In [1]:
# Import Modules
import torch
import numpy as np
import matplotlib.pyplot as plt

# Import your modules
import os
os.chdir('/Users/jingyizhang/Desktop/MA4288O/deephedging/MA4288O-Project')
from market.heston import MultiAssetHestonModel
from optimizer.hedge_train import MultiAssetDeepHedgeCVaRTrainer
from payoff.european_option import multi_asset_european_call_payoff
from utils.networks import RecurrentHedgeModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Simulation Setup
num_assets = 5
num_paths = 100000
num_steps = 30
heston = MultiAssetHestonModel(num_assets=num_assets)
S_paths_np, V_paths_np = heston.simulate_paths(n_paths=num_paths, n_steps=num_steps)

# Compute Payoff
K = 100.0
Z_np = multi_asset_european_call_payoff(S_paths_np, K=K)

# Convert to PyTorch tensors
S_torch = torch.from_numpy(S_paths_np).float().to(device).view(num_paths, num_steps + 1, num_assets)
Z_torch = torch.from_numpy(Z_np).float().to(device)

# Train Model
num_steps = S_torch.shape[1] - 1
recurrent_model_multi = RecurrentHedgeModel(steps=num_steps, hidden_dim=32).to(device)
trainer_multi = MultiAssetDeepHedgeCVaRTrainer(model=recurrent_model_multi, alpha=0.5)

p0_multi = trainer_multi.train(
    S_tensor=S_torch,
    Z_tensor=Z_torch,
    p0_init=0.0,
    n_epochs=5,
    batch_size=4096
)

print(f"Multi-Asset Recurrent Hedge Price p0_multi = {p0_multi:.4f}")

# Compute PnL
with torch.no_grad():
    deltas_multi = recurrent_model_multi(S_torch)
    Sdiff_multi = S_torch[:, 1:, :] - S_torch[:, :-1, :]
    gains_multi = torch.sum(deltas_multi * Sdiff_multi, dim=1)
    pnl_multi = (trainer_multi.p0 - Z_torch + gains_multi).cpu().numpy()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (4096x7 and 3x32)

In [ ]:
# Plot Convergence (Figures 10 & 11)
log_eps = np.log([1e-7, 1e-6, 1e-5, 1e-4, 1e-3])
log_p_diff = np.log(np.abs(p0_multi - Z_np.mean()))

plt.plot(log_eps, log_p_diff, 'ro')
plt.plot(log_eps, 0.7 * log_eps + log_p_diff[0], 'b-')  # Approximate convergence line
plt.xlabel("log(ε)")
plt.ylabel("log(pε - p0)")
plt.title("Rate of Convergence")
plt.show()